In [38]:
%pip install -q python-dotenv langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [39]:
from dotenv import load_dotenv

load_dotenv()

True

In [40]:
from langchain_openai import ChatOpenAI

query = "What is the capital of South Korea?"

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm.invoke(query)

AIMessage(content='The capital of South Korea is Seoul.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C7gw00bCIYc1Log6eM5tSZpTCSCtp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4f1c0024-c54c-4e6d-8a90-1d5cbcc2ff50-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- State : 상태
- Node : Agent 가 수행하는 작업
- Edge : 노드들 사이의 관계
- Conditional Edge :  조건부 관계 

In [41]:
from typing import TypedDict
from typing_extensions import Annotated

from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [42]:
from langgraph.graph import StateGraph

graph = StateGraph(AgentState)

In [43]:
def generate(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

In [44]:
from langgraph.graph import START, END

graph.add_node("generate", generate)
graph.add_edge(START, "generate")
graph.add_edge("generate", END)
graph = graph.compile()

In [45]:
from langchain_core.messages import HumanMessage

initial_state = {"messages": [HumanMessage(content=query)]}
graph.invoke(initial_state)

{'messages': [HumanMessage(content='What is the capital of South Korea?', additional_kwargs={}, response_metadata={}, id='42dd3736-577e-4f87-97be-d49a74defea9'),
  AIMessage(content='The capital of South Korea is Seoul.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C7gw1cGVfrgBYR7c7BrY2dh5aJOeB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c6e73a4b-b626-4b18-b055-cd7b6d2f3d53-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}